In [ ]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

# The digits dataset
digits = datasets.load_digits()

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)

In [ ]:
import joblib

model_file_name = "mnist-svm.joblib"
joblib.dump(classifier, model_file_name)

In [ ]:
from mlserver.grpc.dataplane_pb2 import ModelInferRequest

req = ModelInferRequest()
req.model_name = "mnist-svm"
req.model_version = "v0.1.0"
req

In [ ]:
from mlserver.grpc.converters import  ModelInferRequestConverter
from mlserver.types import InferenceRequest
import json

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": x_0.shape,
          "datatype": "FP32",
          "data": x_0.tolist()
        }
    ]
}

req_t = InferenceRequest(**inference_request)
ModelInferRequestConverter.from_types(req_t, model_name="v0.1.0", model_version="mnist-svm")

In [ ]:
import requests

x_0 = X_test[0:1]
inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": x_0.shape,
          "datatype": "FP32",
          "data": x_0.tolist()
        }
    ]
}

endpoint = "http://localhost:8080/v2/models/mnist-svm/versions/v0.1.0/infer"
response = requests.post(endpoint, json=inference_request)

response.json()